In [0]:
from pyspark.sql.types import StringType,IntegerType,DoubleType,TimestampType,StructType,StructField
from pyspark.sql.functions import col
from pyspark.sql import functions as sf

In [0]:
%run ../configurations/paths_config

In [0]:
%run ../utils/etl_support_functions

In [0]:
dbutils.widgets.text("File Date","2021-03-21")
v_file_date = dbutils.widgets.get("File Date")

In [0]:
new_schema = StructType(fields=[StructField("raceId",IntegerType(),False),
                            StructField("driverId",IntegerType(),True),
                            StructField("lap",IntegerType(),True),
                            StructField("position",IntegerType(),True),
                            StructField("time",StringType(),True),
                            StructField("milliseconds",IntegerType(),True)])

In [0]:
raw_df = spark.read.schema(new_schema).json(raw_path+f"/{v_file_date}/lap_times")

In [0]:
renamed_df = raw_df.withColumnRenamed("raceId","race_id").withColumnRenamed("driverId","driver_id")

In [0]:
audited_df = renamed_df.withColumn("ingestion_time",sf.current_timestamp())

In [0]:
merge_condition = 'tgt.race_id = upd.race_id AND tgt.driver_id = upd.driver_id AND tgt.lap = upd.lap'
upsert_into_delta_table(audited_df, 'vsarthicat.formula1_silver.lap_times',merge_condition,'race_id')